# 데이콘 Basic Summer
## 서울 랜드마크 이미지 분류 경진대회


# I. 데이터 살펴보기

## 1. 데이터 준비
### 1.1 csv 데이터

In [1]:
# pandas 사용하여 데이터 불러오기

import pandas as pd
label_df = pd.read_csv('dataset/train.csv')
label_df.head()

,file_name,label
0,001.PNG,9
1,002.PNG,4
2,003.PNG,1
3,004.PNG,1
4,005.PNG,6


### 1.2 이미지 데이터

데이터 이미지의 local address와 label 값을 list에 저장

In [2]:
import os
from glob import glob
# CNN 모델에서 꼭 필요한 두가지 모듈
# 'glob' : 파일의 경로명을 리스트로 뽑을 때 사용

def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir,'*.PNG')))
    img_path_list = list(map(lambda x: x.replace('\\','/', 10), img_path_list))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))

    # get label
    label_list.extend(label_df['label'])
    
    #print(img_path_list)
    #print(label_list)
    
    return img_path_list,label_list

def get_test_data(data_dir):
    img_path_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir,'*.PNG')))
    img_path_list = list(map(lambda x: x.replace('\\','/', 10), img_path_list))
    img_path_list.sort(key=lambda x: int(x.split('/')[-1].split('.')[0]))
    #print(img_path_list)
    
    return img_path_list

## 2. 데이터 확인
### 2.1. csv 데이터

pandas의 `info()` 메소드 활용하여 데이터의 특성 살펴보기

In [3]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  723 non-null    object
 1   label      723 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.4+ KB


### 2.2. 이미지 데이터

In [4]:
all_img_path, all_label = get_train_data('dataset/train')
test_img_path = get_test_data('dataset/test')

#['dataset/train/001.PNG',
#['dataset/train\\001.PNG',

In [5]:
all_label[:5]

[9, 4, 1, 1, 6]

In [6]:
all_img_path[:5]

['dataset/train/001.PNG',
 'dataset/train/002.PNG',
 'dataset/train/003.PNG',
 'dataset/train/004.PNG',
 'dataset/train/005.PNG']

In [7]:
test_img_path[:5]

['dataset/test/001.PNG',
 'dataset/test/002.PNG',
 'dataset/test/003.PNG',
 'dataset/test/004.PNG',
 'dataset/test/005.PNG']

## 3. 환경설정
데이터를 전처리 하기위함 GPU 딥러닝 환경설정

In [8]:
import torch
import torch.nn as nn

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVISE_ORDER"] = "PCI_BUS_ID"  # rrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # Set the GPU 2 to use, 멀티 GPU

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
torch.cuda.is_available()

False

In [10]:
# GPU 체크 및 할당

if torch.cuda.is_available():
    #device = torch.device("cuda:0")
    print("Device:",device)
    print("There are %d GPU(s) available"%torch.cuda.device_count())
    print("We will use the GPU:",torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU availavle, using the CPU instead.")

No GPU availavle, using the CPU instead.


# 똥컴이라 안됨

In [11]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

1.12.1
False


In [12]:
# 하이퍼 파라미터 튜닝
CFG = {
    'IMG_SIZE':128, # 이미지 사이즈
    'EPOCHS':50, # 에포크 - 전체 데이터를 사용하여 학습하는 횟수
    'LEARNING_RATE':2e-2, # 학습률
    'BATCH_SIZE':12, #배치사이즈
    'SEED':41, #시드
}

In [13]:
# 모델의 재현성을 위해 random seed 고정

import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #torch.bakcends.cudnn.deterministic = True
    #torch.backends cudnn benchmark = True
    
seed_everything(CFG['SEED'])

# II. 데이터 전처리
## 2. CustomDataset
CustomDataset을 만들어 전체 dataset을 구성

In [14]:
import torchvision.datasets as datasets  # 이미지 데이터셋 집합체
import torchvision.transforms as transforms  # 이미지 변환 툴 - 텐서 변환, 이미지 정규화...

from torch.utils.data import DataLoader  # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

import cv2   # 이미지 출력하고, 저장하고, 새 윈도우에 보여주고 등등

class CustomDataset(Dataset):
    #필요한 변수 선언
    def __init__(self,img_path_list,label_list,train_mode = True, transforms = None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list
    
    # index번째 data를 return
    def __getitem__(self,index):
        img_path = self.img_path_list[index]  # 이미지 파일 경로
        print(img_path)
        
        # Get image data
        image = cv2.imread(img_path)  # 이미지 파일
        if self.transforms is not None:   # transfrom 형태가 주어지면 transfrom하여 저장
            image = self.transforms(image)
            
        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):  # 길이 출력
        return len(self.img_path_list)
            

In [15]:
# 임시 데이터셋 만들어 정상 작동하는지 확인
tempdataset = CustomDataset(all_img_path, all_label,train_mode = False)

In [16]:
import matplotlib.pyplot as plt

#plt.imshow(tempdataset.__getitem__(0))

# Kernal Died ?? Prolly GPU 사용 못해서 메모리 사용량 과다로 죽은듯

## 3. Train / Validation Split

학습시킬 데이터셋과 검증할 데이터셋 분리!

In [17]:
# Train : Validation = 0.75: 0.25 Split

train_len = int(len(all_img_path)*0.75)
Vali_len = int(len(all_img_path)*0.25)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

In [18]:
print("train set 길이:", train_len)
print("validation set 길이:", Vali_len)

train set 길이: 542
validation set 길이: 180


## 4. Transfrom

나뉜 데이터셋에서 이미지를 분석하기 위해 이미지 변형(transform) 적용

In [19]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),  # Numpy 배열에서 PIL 이미지로 변형
    transforms.Resize([CFG['IMG_SIZE'],CFG['IMG_SIZE']]), # 이미지 사이즈 변형
    transforms.ToTensor(), #이미지 데이터를 tensor
    transforms.Normalize(mean= (0.5,0.5,0.5),std = (0.5,0.5,0.5)) # 이미지 정규화
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]),\
    transforms.ToTensor(),
    transforms.Normalize(mean =(0.5,0.5,0.5),std = (0.5,0.5,0.5))
])

## 5. Dataloader
- `Dataloader` Class 생성: batch 기반 딥러닝모델 학습을 위해 mini batch를 만들어주는 역할

- dataset의 전체 데이터가 batch size로 나뉨

- 만들었던 dataset을 input으로 넣어주면 여러 옵션(데이터 묶기, 섞기, 알아서 병렬처리)를 통해 batch 생성

In [20]:
# Get Dataloader

# CustomDataset class를 통하여 train dataset 생성
train_dataset = CustomDataset(train_img_path, 
                              train_label, 
                              train_mode = True, 
                              transforms = train_transform)
# 만든 train dataset를 DataLoader에 넣어 batch 만들기
train_loader = DataLoader(train_dataset,
                          batch_size = CFG['BATCH_SIZE'],
                          shuffle=True, 
                          num_workers =0)


# Validation 에서도 적용
vali_dataset = CustomDataset(vali_img_path, 
                             vali_label, 
                             train_mode = True, 
                             transforms = test_transform)
vali_loader = DataLoader(vali_dataset,
                         batch_size = CFG['BATCH_SIZE'],
                         shuffle=True, 
                         num_workers =0)

In [21]:
train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('total train imgs: ',train_len,'/ total train batches(iterations) :',train_batches)
print('total valid imgs: ',Vali_len,'/ total valid batches(iterations) :',vali_batches)

total train imgs:  542 / total train batches(iterations) : 46
total valid imgs:  180 / total valid batches(iterations) : 16


**train 데이터의 경우, 542 장의 이미지를, 12개짜리 배치로 나누어 1번의 epoch를 돌기 위해 46개의 batch를 반복 학습해야한다.**

- 배치 사이즈 : 12 
- train - 46 묶음 / validation - 16 묶음

# III. 모델링
## 1. 모델 구조 정의
CNN모델을 이해하기 위한 기본 개념

### - Neural Network (신경망)
신경망은 다른 말로 인공 신경망이라고도 불린다.

신경망이란 뇌 속 뉴런의 망형 구조를 닮은 다층형 구조의 컴퓨팅 모델이다.
여기에서 서로 연결된 처리 소자, 일명 '뉴런'이라는 것이 있으며 생물학적 뉴런을 수학적으로 모델린한 것인데, 이들이 서로 협력하여 출력 함수를 도출한다.

신경망은 입력 및 출력 계층/차원으로 구성되며 대부분은 숨겨진 계층으로 이루어져 있다. 숨겨진 계층은 입력을 출력 계층에서 사용할 수 있는 무언가로 변환해주는 단위로 구성된다. 따라서, 인공신경망 뉴런은 여러 입력값을 받아 일정 수준을 넘어서게 되면 활성화되고 출력값을 내보낸다.

### 1) Activation Function (활성화 함수)

딥러닝 네트워크에서 노드에 입력된 값들을 비선형 함수에 통과시킨 후 다음 레이어로 전달하는 데, 이 떄 사용하는 함수를 활성화 함수(Activation Function)이라 한다. 

선형 함수가 아니라 비선형 함수를 사용하는 이유는 딥러닝 모델의 레이어 층을 깊게 가져갈 수 있기 때문이다.

**대표적인 활성화 함수 살펴보기**

#### 1.1) Sigmoid 함수

Sigmoid 함수는 Logistic 함수라고 불리기도 하며, x의 값에 따라 0~1의 값을 출력하는 S자형 함수이다.

`Binary Classification` 마지막 레이어의 활성화 함수로 사용된다.

Sigmoid 함수는 Activation function으로 많이 사용되며, 보통 0.5 미만은 0, 이상은 1을 출력한다.

$$sigmoid(x)\\
= 1 / (1 + exp(-x))\\
= exp(x) / (exp(x) + 1)$$

![시그모이드함수](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/480px-Logistic-curve.svg.png)

그래프를 보면, 양 극단 값의 기울기(미분 값)이 0에 가까워져, 학습이 되지 않는 문제가 발생한다.

이를 **Gradient Vanishing 문제**라고 하는데 이러한 문제를 해결하기 위해 다양한 활성화함수들이 고안되었다.

### 2) Perceptron (퍼셉트론)

퍼셉트론이란 신경망의 기원이 되는 개념으로, Frank Rosenblatt가 1957년에 고안한 알고리즘이다. 

이는 여러 신호를 입력받아 일정한 임계값을 넘는지를 확인하여, 0(흐르지 않는다, 임계값을 넘지 않았다) 또는 1(흐른다, 임계값을 넘었다)라는 출력값을 앞으로 전달한다.

**(뉴런의 all-or-none 늒김)**

각 입력 신호 X는 각 가중치 w와 곱해진다. 가중치가 클수록 그 신호가 중요하다는 뜻이다.

그러나, 퍼셉트론은 단순한 선형 분류기로, AND나 OR과 같은 분류는 가능하나, XOR 분류는 불가능하다.

![퍼셉트론 러닝 알고리즘](https://image.slidesharecdn.com/machine-learning-120930145310-phpapp01/95/machine-learning-with-applications-in-categorization-popularity-and-sequence-labeling-75-638.jpg?cb=1354541953)


**편향(bias) = $theta$ (임계치)** 

: 노드를 얼마나 쉽게 활성화(1로 출력; activation) 되느냐를 조정하는 매개변수
* theta (임계치)가 높아질수록, 엄격한 모델이 되며, underfitting의 문제 발생 가능
* theat (임계치)가 낮아질수록, 느슨한 모델이 되며, overfitting, 과도한 noise 포함 등의 문제 발생 
